# COMPSCI 761 22S2 Assignment 1

- Lecturer: Anna Trofimova
- School of Computer Science, The Univerity of Auckland
- Last update 29th of July at 18:00pm, 2022
$$
% macros
\newcommand{\indep}{\perp \!\!\!\perp}
$$

## Student:

You're favourite student; **Florian Suess, fsue534**

### Submission
This interactive notebook contains the instructions to complete assignment 1; You should submit this notebook with the code and answers in one single file in .ipybn format with name assignment1.ipybn. **Write your name and UPI in the cell above** (to edit the markdown text double-click the cell).

There is a maximum file size cap of 5MB, so make sure your submission does not exceed this size. The submitted notebook should contain all your source code and answers. You can add new cells and use markdown text to organise and explain your implementation/answer.

Submit your files using CANVAS upload function.

The submission deadline is **19th of August at 11.59pm, 2022.**

### Plagiarism
This is an individual assignment. Remember that **all** work submitted for this assignment must be your own **individual** work and no code
sharing or copying is allowed. You may use code from the Internet only with suitable attribution
of the source in your program. **Do not use public code repositories as your code might be copied.** Keep in mind that sharing parts of assignment solutions is a form of plagiarism. All submitted assignments will be run through plagiarism detection software to detect similarities to other submissions. You should carefully read the UoA policy on academic integrity and plagiarism. 

### Description

This assignment consists of two problems presented in a puzzle game context. For the first problem, you will be implementing a path search problem along with common search algorithms and evaluating their properties. For the second problem, you will be exploring how to define and solve a constraint satisfaction problem. To implement the solutions, you have to use AIPython code that provides you with many implemented classes and examples of their use.  

When working with a Jupyter notebook, you can edit the \*.py files either in the Jupyter interface (in your browser) or with your favorite editor (e.g., PyCharm). Whenever you save a \*.py file, the notebook will reload their content directly.

The libraries that you can use (and need) in this assignment are  the following (run the cell below to import the libraries):

In [1]:
import sys
sys.path.insert(0, "./aipython")
import numpy as np
from aipython.searchGeneric import *
from aipython.searchProblem import *
from aipython.cspProblem import *
from aipython.cspSearch import *

"""as per https://canvas.auckland.ac.nz/courses/72095/discussion_topics/1126883"""
import time
import copy
import operator


import signal

class timeout:
    """Utility to contain the run time of algorithms programmatically, allows
    for clearer cell output logging for assignment marker. Throws Timeout exception
    after 5min"""
    def handle_timeout(self, signum, frame):
        raise TimeoutError()
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout) # listens for SIGALRM signal; call handle_timeout()
        signal.alarm(15) # schedules the emittion of a SIGALRM after 5 minutes TODO
    def __exit__(self, type, value, traceback):
        signal.alarm(0) # cancels scheduled SIGALRM

def begin_search(searcher):
    try:
        with timeout():
            searcher.search()
    except TimeoutError:
        print("... TIMEOUT", searcher.num_expanded, "paths have been expanded and", 
            len(searcher.frontier), "paths remain in the frontier")

*Tip: If you placed the notebook correctly it should not give you any errors.*

*If you think that there are other libraries you might need then send an email to anna.trofimova@auckland.ac.nz to confirm.*

### Part 1 - Heuristics [5 marks]

This part of the assignment is based on a popular puzzle, Game of Fifteen, also known as 15-puzzle. The game consists of 15 numbered tiles on a four by four grid where one tile is missing. To solve the puzzle, the tiles must be moved so that they are ordered from 1 to 15. The goal state of the puzzle is defined as a multidimensional array of digits#, where 0 represents the missing tile, as follow:

In [2]:
goal = [[1, 2, 3, 4], 
        [5, 6, 7, 8], 
        [9, 10, 11, 12], 
        [13, 14, 15, 0]]

#### Task 1.1
Edit the code below to implement a search problem class representing the Game of Fifteen that uses heuristic based on Manhattan Distance (h1). The class must be implemented by extending the class Search\_problem. 

Manhattan Distance between two points $p1$ at $(x1, y1)$ and $p2$ at $(x2, y2)$ is $d_{M}(p1, p2) = |x1 - x2| + |y1 - y2|$

*Tip: If you have problems understanding how to overwrite methods take a look at the implementation of Search_problem_from_explicit_graph class in searchProblem.py*


In [3]:

"""
    Intuition; a board with it's configuration of tiles is a node in my search tree. The neighbours of such board 
    neighbours of this node being all board configurations that can be derived by 1 move; defined by the legal moves,
    that is, the moving of the below/above/left/right tile into the empty slot (at most 4 neighbours).
"""

class GameFifteenProblem(Search_problem):
    ROWS=4
    COLUMNS=4
            
    def __init__(self, start: list[list[int]], goal: list[list[int]]) -> None:
        """Sets problem, will throw ValueError if board and/or goal given isn't to spec."""
        
        def must_be_valid_board(board: list[list[int]]) -> None:
            values: list[int] = []
            if len(board) != self.ROWS:
                raise ValueError(f"the fifteen game boards provided must have exactly {self.ROWS} rows")
            for row in board:
                if len(row) != self.COLUMNS:
                    raise ValueError(f"the fifteen game boards provided must have exactly {self.COLUMNS} columns")
                for value in row:
                    values.append(value)
            
            values.sort()
            if not np.array_equal(values, [i for i in range(16)]):
                raise ValueError(f"expect only 0,1,...,15 tiles as board input")

        must_be_valid_board(start)
        must_be_valid_board(goal)
        
        self.start_tiles: list[list[int]] = start
        self.goal_tiles: list[list[int]] = goal
        return

 

    def start_node(self) -> list[list[int]]:
        """Returns the start node, which in this case is the starting board configuration."""
        return self.start_tiles
    
    def is_goal(self, board: list[list[int]]) -> bool:
        """Returns True if the given board (list of tiles) is the goal, otherwise False."""
        for y, goal_row in enumerate(self.goal_tiles):
            for x, goal_value in enumerate(goal_row):
                if board[y][x] != goal_value:
                    return False
        
        return True
    
    def neighbors(self, board: list[list[int]]) -> list[Arc]:
        """Returns a list of the arcs for the neighbors of board, note the neighbours are
        at most four board configurations that are based on swaps with the 0 tile."""
        
        def swap(board: list[list[int]], zero: tuple[int, int], tile: tuple[int, int]) -> list[list[int]]:
            """Takes in a board, top left zero indexed (x,y) tuples for candidate titles.
            Returns a new board with zero and tile swapped."""
            swapped_board = copy.deepcopy(board)
            
            swapped_board[zero[1]][zero[0]] = board[tile[1]][tile[0]]
            swapped_board[tile[1]][tile[0]] = 0
                
            return swapped_board
        
        neighbours: list[Arc] = []
        for y, row in enumerate(board):
            for x, value in enumerate(row):
                if value == 0:
                    """can only ever swap the 0 tile with it's immediate neighbours"""
                    zero_tile = (x, y)
                    
                    if y != 0:
                        zero_tile_swapped_up = swap(board, zero_tile, (x, y-1))
                        neighbours.append(Arc(board, zero_tile_swapped_up))
                        
                    if y + 1 != self.ROWS: 
                        zero_tile_swapped_down = swap(board, zero_tile, (x, y+1))
                        neighbours.append(Arc(board, zero_tile_swapped_down))
                        
                    if x != 0: 
                        zero_tile_swapped_left = swap(board, zero_tile, (x-1, y))
                        neighbours.append(Arc(board, zero_tile_swapped_left))
                        
                    if x + 1 != self.COLUMNS: 
                        zero_tile_swapped_right = swap(board, zero_tile, (x+1, y))
                        neighbours.append(Arc(board, zero_tile_swapped_right))
                    
        return neighbours
    
    def heuristic(self, board: list[list[int]]) -> int:
        """Returns the heuristic value of the node based on the sum of the Manhattan distances
        of each current tile positions to their respective desired goal position."""
        
        heuristic: int = 0
        def get_goal_position(target: int) -> tuple[int, int]:
            for y, row in enumerate(self.goal_tiles):
                for x, value in enumerate(row):
                    if value == target:
                        return (x, y)

        for y, row in enumerate(board):
            for x, value in enumerate(row):
                if value == 0:
                    continue
                goal_position = get_goal_position(value)
                heuristic += ( abs(x-goal_position[0]) + abs(y-goal_position[1]) )
                
        return heuristic

To validate the correctness of the problem class use the A* searcher algorithm (from searchGeneric.py) to find a solution. 

*Tip: The cost of the solution should be 9.*

In [4]:
start = [[1, 2, 3, 4], 
         [9, 5, 6, 7], 
         [10, 11, 8, 0], 
         [13, 14, 15, 12]]

puzzle = GameFifteenProblem(start, goal)
searcher = AStarSearcher(puzzle)
solution = searcher.search()
print('Cost: ',  solution.cost)

11 paths have been expanded and 23 paths remain in the frontier
Cost:  9


#### Task 1.2 
Implement search problem classes representing the Game of Fifteen that use heuristics based on Euclidean Distance (h2) and the number of the inversions of the permutation (h3). The classes must be implemented by extending the class GameFifteenProblem.

Euclidean distance between two points $p1$ at $(x1, y1)$ and $p2$ at $(x2, y2)$ is $d_{E}(p1, p2) =  \sqrt{(x1-x2)^{2}+ (y1-y2)^{2}}$

An inversion of a permutation $(t_{1},t_{2},...,t_{n})$ of the elements in a finite n-element set of positive integers $A=\{1,2,...,n\}$ is the pair $(t_{j},t_{k})$, where $j<k$ and $t_{j}>t_{k}$.

$\sum_{j=1}^{N} \sum_{k=j}^{N}    \{t_{j} > t_{k}: 1, otherwise: 0\}$, where $N$ is the total number of elements and $t_{i}$ is the value of i-th element.

In [5]:
class GameFifteenProblemEuclidean(GameFifteenProblem):
    def __init__(self, start: list[list[int]], goal: list[list[int]]) -> None:
        (super().__init__(start, goal))

    def heuristic(self, board: list[list[int]]) -> int:
        heuristic: int = 0
            
        def get_goal_position(target: int) -> tuple[int, int]:
            for y, row in enumerate(self.goal_tiles):
                for x, value in enumerate(row):
                    if value == target:
                        return (x, y)

        for y, row in enumerate(board):
            for x, value in enumerate(row):
                if value == 0:
                    continue
                goal_position = get_goal_position(value)
                
                heuristic += ( (x-goal_position[0])**2 + (y-goal_position[1])**2 )**(1/2)
                
        return heuristic

In [6]:
class GameFifteenProblemInversions(GameFifteenProblem):
    def __init__(self, start, goal):
        (super().__init__(start, goal))

    def heuristic(self, board: list[list[int]]) -> int:
        heuristic: int = 0
            
        flattened_board: list[int] = []
        for row in board:
            for value in row:
                flattened_board.append(value)
        
        for j in range(16):
            for k in range(j, 16):
                left, right = flattened_board[j], flattened_board[k]
                if left == 0 or right == 0:
                    continue

                if left > right:
                    heuristic += 1

        return heuristic

#### Task 1.3
Run A* Search algorithm with every heuristic for the following three start states:

In [7]:
# optimal path cost: 14
start14 = [[1, 2, 8, 3],
           [5, 6, 7, 4],
           [9, 15, 14, 11],
           [13, 10, 12, 0]]

# optimal path cost: 17
start17 = [[1, 3, 6, 4],
           [5, 2, 8, 14],
           [9, 15, 7, 0],
           [13, 10, 12, 11]]

# optimal path cost: 23
start23 = [[1, 3, 6, 4],
           [5, 8, 15, 14],
           [9, 2, 7, 0],
           [13, 10, 12, 11]]

for start in [start14, start17, start23]:
    print('Starting Position: ', start)
    print('=========')

    print('- Manhattan Heuristic Method: ')
    problem = GameFifteenProblem(start, goal)
    searcher = AStarSearcher(GameFifteenProblem(start, goal))
    begin_search(searcher)
    print ("  Heuristic value of start state: ", problem.heuristic(start))
    if hasattr(searcher, 'solution'): print("  Cost: ", searcher.solution.cost)
    
    print('- Euclidean Heuristic Method: ')
    
    problem = GameFifteenProblemEuclidean(start, goal)
    searcher = AStarSearcher(problem)
    begin_search(searcher)
    print ("  Heuristic value of start state: ", problem.heuristic(start))
    if hasattr(searcher, 'solution'): print("  Cost: ", searcher.solution.cost)
    
    print('- Inversions Heuristic Method: ')
    
    problem = GameFifteenProblemInversions(start, goal)
    searcher = AStarSearcher(problem)
    begin_search(searcher)
    print ("  Heuristic value of start state: ", problem.heuristic(start))
    if hasattr(searcher, 'solution'): print("  Cost: ", searcher.solution.cost)

Starting Position:  [[1, 2, 8, 3], [5, 6, 7, 4], [9, 15, 14, 11], [13, 10, 12, 0]]
- Manhattan Heuristic Method: 
45 paths have been expanded and 85 paths remain in the frontier
  Heuristic value of start state:  12
  Cost:  14
- Euclidean Heuristic Method: 
196 paths have been expanded and 390 paths remain in the frontier
  Heuristic value of start state:  9.65685424949238
  Cost:  14
- Inversions Heuristic Method: 
30 paths have been expanded and 62 paths remain in the frontier
  Heuristic value of start state:  20
  Cost:  14
Starting Position:  [[1, 3, 6, 4], [5, 2, 8, 14], [9, 15, 7, 0], [13, 10, 12, 11]]
- Manhattan Heuristic Method: 
28 paths have been expanded and 61 paths remain in the frontier
  Heuristic value of start state:  17
  Cost:  17
- Euclidean Heuristic Method: 
309 paths have been expanded and 704 paths remain in the frontier
  Heuristic value of start state:  13.485281374238573
  Cost:  17
- Inversions Heuristic Method: 
1029 paths have been expanded and 2422 pat

In each case record in the tables below the heuristic values for the start states, the number of the expanded nodes, and the costs of the solutions.

| Heuristic | h-value: start14 | h-value: start17  | h-value: start23  |
|-----------|------------------|-------------------|-------------------|
|Manhattan  | 12               | 17                | 19                | 
|Euclidean  | 9.65685424949238 | 13.485281374238573| 16.071067811865476| 
|Inversions | 20               | 23                | 29                | 

| Heuristic | expanded: start14 | expanded: start17 | expanded: start23 |
|-----------|-------------------|-------------------|-------------------|
|Manhattan  | 45                | 28                | 617               | 
|Euclidean  | 196               | 309               | 10083             |  
|Inversions | 30                | 1029              | 87774             |  

| Heuristic | path cost: start14 | path cost: start17 | path cost: start23 |
|-----------|--------------------|--------------------|--------------------|
|Manhattan  | 14                 | 17                 | 23                 | 
|Euclidean  | 14                 | 17                 | 23                 |
|Inversions | 14                 | 17                 | 27                 |

Comment on the performance of the A* search algorithm with each heuristic based on the results in the tables. Explain which heuristic is better/worse and why.

### Part 2:  Search algorithms [7 marks]

*Tip: If you have problems understanding how to overwrite methods take a look at the implementation of AStarSearcher and  Searcher classes in searchGeneric.py*

*Tip: To initialize the frontier think of the type of structure used by the algorithm to store generated nodes. If it uses a priority queue use FrontierPQ from serachGeneric.py*

#### Task 2.1
Implement a class that performs Breadth First Search by extending class Searcher.

In [8]:
class BreadthFirstSearcher(Searcher):
    def __init__(self,  problem: Search_problem) -> None:
        super().__init__(problem)
    
    def initialize_frontier(self) -> None:
        self.frontier = []
    
    def empty_frontier(self) -> bool:
        return self.frontier == []

    def search(self) -> None:
        while not self.empty_frontier():            
            path = self.frontier.pop(0) # flip this to pop() and you've got yourself a DFS searcher
            self.display(2, "Expanding:",path,"(cost:",path.cost,")")
            self.num_expanded += 1
            if self.problem.is_goal(path.end()):
                self.display(1, self.num_expanded, "paths have been expanded and", 
                             len(self.frontier), "paths remain in the frontier")
                self.solution = path
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(path,arc))
                self.display(3,"Frontier:",self.frontier)
        self.display(1,"No (more) solutions. Total of",self.num_expanded,"paths expanded.")

#### Task 2.2

Implement a class that performs Iterative Deepening Search by extending class Searcher.

In [9]:
class IterativeDeepeningSearcher(Searcher):
    def __init__(self, problem: Search_problem) -> None:
        super().__init__(problem)
        self.iterative_depth = 0
        self.exceeded_threshold: list[Path] = []

    def initialize_frontier(self) -> None:
        self.frontier = []

    def empty_frontier(self) -> bool:
        return self.frontier == []

    def add_to_frontier(self, path) -> None:
        self.frontier.append(path)
    
    def search(self) -> None:
        while not self.empty_frontier():
            path = self.frontier.pop()
            self.display(2, "Expanding:",path,"(cost:",path.cost,")")
            self.num_expanded += 1
            
            if path.cost > self.iterative_depth:
                self.display(2, "Cannot continue, hit iterative depth limit")
                self.exceeded_threshold.append(path)
                continue
            elif self.problem.is_goal(path.end()):
                self.display(1, self.num_expanded, "paths have been expanded and", 
                             len(self.frontier), "paths remain in the frontier")
                self.solution = path
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(path,arc))
                self.display(3,"Frontier:",self.frontier)

        if self.exceeded_threshold == []:
            self.display(1,"No (more) solutions. Total of",self.num_expanded,"paths expanded.")
            return
    
        self.iterative_depth += min(map(lambda path: path.cost, self.exceeded_threshold))
        self.add_to_frontier(Path(self.problem.start_node()))
        self.exceeded_threshold = []
        return self.search()

#### Task 2.3
Implement a class that performs Iterative Deepening A* Search by extending class Searcher.

In [10]:
class IterativeDeepeningAStarSearcher(Searcher):
    def __init__(self, problem: Search_problem) -> None:
        super().__init__(problem)
        self.iterative_f_value_depth = self.problem.heuristic(problem.start_node())
        self.exceeded_threshold: list[Path] = []
            
    def initialize_frontier(self) -> None:
        self.frontier = FrontierPQ()

    def empty_frontier(self) -> bool:
        return self.frontier.empty()

    def add_to_frontier(self,path) -> None:
        self.frontier.add(path, path.cost + self.problem.heuristic(path.end()))
    
    def search(self) -> None:
        while not self.empty_frontier():
            path = self.frontier.pop()
            f_value = path.cost + self.problem.heuristic(path.end())
            self.display(2, "Expanding:",path,"(cost:",path.cost,",f_value:",f_value,")")
            self.num_expanded += 1
            
            if f_value > self.iterative_f_value_depth:
                self.display(2, "Cannot continue, hit iterative f value depth level")
                self.exceeded_threshold.append(path)
                continue
            elif self.problem.is_goal(path.end()):
                self.display(1, self.num_expanded, "paths have been expanded and", 
                             len(self.frontier), "paths remain in the frontier")
                self.solution = path
                return path
            else:
                neighs = self.problem.neighbors(path.end())
                self.display(3,"Neighbors are", neighs)
                for arc in reversed(list(neighs)):
                    self.add_to_frontier(Path(path,arc))
                self.display(3,"Frontier:",self.frontier)

        if self.exceeded_threshold == []:
            self.display(1,"No (more) solutions. Total of",self.num_expanded,"paths expanded.")
            return
        
        self.iterative_f_value_depth = min(map(lambda path: path.cost + self.problem.heuristic(path.end()), self.exceeded_threshold))
        self.add_to_frontier(Path(self.problem.start_node()))
        self.exceeded_threshold = []
        return self.search()

#### Task 2.4
Implement a class that performs Uniform Cost Search by extending class Searcher.

In [11]:
class UniformCostSearcher(Searcher):
    def initialize_frontier(self):
        self.frontier = FrontierPQ()

    def empty_frontier(self):
        return self.frontier.empty()

    def add_to_frontier(self, path):
        self.frontier.add(path, path.cost + self.problem.heuristic(path.end()))

#### Task 2.5

Run Breadth First Search (BFS), Iterative Deepenining Search (IDS), Iterative Deepening A* Search (IDA\*S), and Uniform Cost Search (UCS) algorithms on each of the 5 start states:

In [12]:
# optimal path cost: 10
start10 = [[2, 3, 7, 4],
           [1, 6, 11, 8],
           [5, 10, 0, 12],
           [9, 13, 14, 15]]

# optimal path cost: 24
start24 = [[2, 7, 11, 4],
           [6, 3, 12, 0],
           [1, 5, 15, 8],
           [9, 10, 13, 14]]

# optimal path cost: 30
start30 = [[2, 7, 11, 4],
           [6, 3, 12, 0],
           [1, 5, 15, 14],
           [9, 10, 8, 13]]

# optimal path cost: 36
start36 = [[7, 11, 12, 4],
           [2, 3, 14, 1],
           [6, 5, 13, 8],
           [9, 10, 15, 0]]

# optimal path cost: 41
start41 = [[7, 11, 12, 4],
           [2, 3, 8, 14],
           [10, 0, 5, 1],
           [6, 9, 13, 15]]

for start in [start10, start24, start30, start36, start41]:
    print('Starting Position: ', start)
    print('=========')
    print('- Breadth First Search: ', end = '')
    begin_search(BreadthFirstSearcher(GameFifteenProblem(start, goal)))
    print('- A Star Search: ', end = '')
    begin_search(AStarSearcher(GameFifteenProblem(start, goal)))
    print('- Iterative Deepening Search: ', end = '')
    begin_search(IterativeDeepeningSearcher(GameFifteenProblem(start, goal)))
    print('- Iterative Deepening A Star Search: ', end = '')
    begin_search(IterativeDeepeningAStarSearcher(GameFifteenProblem(start, goal)))
    print('- Uniform Cost Search: ', end = '')
    begin_search(UniformCostSearcher(GameFifteenProblem(start, goal)))

Starting Position:  [[2, 3, 7, 4], [1, 6, 11, 8], [5, 10, 0, 12], [9, 13, 14, 15]]
- Breadth First Search: ... TIMEOUT 196001 paths have been expanded and 438276 paths remain in the frontier
- A Star Search: 11 paths have been expanded and 20 paths remain in the frontier
- Iterative Deepening Search: 126737 paths have been expanded and 21 paths remain in the frontier
- Iterative Deepening A Star Search: 11 paths have been expanded and 20 paths remain in the frontier
- Uniform Cost Search: 11 paths have been expanded and 20 paths remain in the frontier
Starting Position:  [[2, 7, 11, 4], [6, 3, 12, 0], [1, 5, 15, 8], [9, 10, 13, 14]]
- Breadth First Search: ... TIMEOUT 202610 paths have been expanded and 453041 paths remain in the frontier
- A Star Search: 1154 paths have been expanded and 2307 paths remain in the frontier
- Iterative Deepening Search: ... TIMEOUT 1248818 paths have been expanded and 18 paths remain in the frontier
- Iterative Deepening A Star Search: 1301 paths have be

In each case, record in the table below the number of nodes generated during the search. If the algorithm runs out of memory or the number of nodes in the frontier exceeds 1 million items, just write “Mem” in your table. If the code runs for five minutes without producing output, terminate the process and write “Time” in your table.

*Tip: To edit the table double click the cell below.*

| Algorithm | 10      | 24      | 30      | 36      | 41      |
|-----------|---------|---------|---------|---------|---------|
|BFS        | #       | #       | #       | #       | #       |
|A*         | 11      | 1154    | 1295    | 78748   | #       |
|IDS        | 126737  | #       | #       | #       | #       |
|IDA\*S     | 11      | 1301    | 3079    | 99190   | #       |
|UCS        | 11      | 1154    | 1295    | 78748   | #       |

Discuss the time and space efficiency of these four algorithms, comment on the results in the table.

### Part 3: Deceptive Starting States [3 marks]

#### Task 3.1 

Run IDA* on the starting states below and report the number of the expanded nodes.

In [48]:
start37_1 = [[7, 11, 12, 4],
             [2, 3, 14, 1],
             [6, 5, 13, 0],
             [9, 10, 15, 8]]

start37_2 = [[7, 11, 12, 4],
             [2, 3, 8, 14],
             [6, 0, 1, 15],
             [9, 5, 10, 13]]

for start in [start37_1, start37_2]:
    print('=========')
    print('Starting Position: ', start)
    puzzle = GameFifteenProblem(start, goal)
    searcher = IterativeDeepeningAStarSearcher(puzzle)
    solution = searcher.search()
    print('Cost: ',  solution.cost)

Starting Position:  [[7, 11, 12, 4], [2, 3, 14, 1], [6, 5, 13, 0], [9, 10, 15, 8]]
115348 paths have been expanded and 162507 paths remain in the frontier
Cost:  37
Starting Position:  [[7, 11, 12, 4], [2, 3, 8, 14], [6, 0, 1, 15], [9, 5, 10, 13]]
4476 paths have been expanded and 4976 paths remain in the frontier
Cost:  37


#### Task 3.2

Explain why there is a difference between the number of the expanded nodes for these starting states if their costs of the optimal paths are the same.

### Part 4:  Constraint Satisfaction Problem [5 marks]

This part of the assignment is based on another puzzle called Sudoku. The game consists of 9x9 grid with a few cells filled in with digits. The objective of the game is to fill the remaining cells so that each column, each row, and each of the nine 3×3 sub-grids contain all of the digits from 1 to 9.

The start state of the puzzle is defined as a multidimensional array of numbers, where 0 represents empty cells, for example:

In [16]:
grid = [[9, 5, 0, 8, 2, 7, 3, 0, 0],
        [0, 8, 0, 1, 4, 0, 0, 5, 0],
        [0, 1, 0, 5, 9, 0, 0, 0, 0],
        [8, 3, 0, 0, 0, 0, 0, 7, 5],
        [1, 6, 9, 7, 5, 2, 4, 3, 0],
        [0, 7, 0, 0, 8, 0, 0, 6, 0],
        [0, 9, 1, 0, 6, 0, 8, 4, 0],
        [7, 0, 8, 0, 3, 1, 0, 0, 6],
        [6, 2, 0, 4, 7, 8, 0, 9, 0]]

*Tip: If you have problems with understanding how to implement constrains take a look at cspExamples.py and cspExamplesQueens.py*

#### Task 4.1

Write the code that defines constraint(s) and a function grid_to_csp that returns a CSP for a Sudoku puzzle:

In [15]:
def grid_to_csp(grid):
    """returns a soduko constraint satisfaction problem"""
    
    def cross(A,B): return [a+b for a in A for b in B]
    rows, cols = 'ABCDEFGHI', '123456789'
    
    """a unit generalises cells that are collections defined by row, column or box"""
    unit_list = (
            [cross(rows, col) for col in cols] +
            [cross(row, cols) for row in rows] +
            [cross(box_rows, box_cols) for box_rows in ('ABC','DEF','GHI') for box_cols in ('123','456','789')]
    )

    domains = {}
    for row_index, row in enumerate(grid):
        for col_index, value in enumerate(row):
            square_index = rows[row_index] + cols[col_index]

            match value:
              case 0: # we interpret zero's as being anything
                  domains[square_index] = Variable(square_index, {1, 2, 3, 4, 5, 6, 7, 8, 9})
              case _: # anything else is the starter value hence reducing the domain to just that value
                  domains[square_index] = Variable(square_index, {value})

    constraints = []
    for square_index, variable in domains.items():
        must_be_different_to = set()
        for unit in [unit for unit in unit_list if square_index in unit]:
            must_be_different_to = must_be_different_to | set([cell for cell in unit])
        must_be_different_to.remove(square_index)
                
        def not_equal(x,y): return x != y
        for square in must_be_different_to:
            constraints.append(Constraint([variable, domains[square]], not_equal))

    
    return CSP("soduku", {variable for _, variable in domains.items()}, constraints)

#### Task 4.2

Write the code that solves Sudoku csp using a search algorithms of your choice, print the solution.

In [42]:
class Backtracking_Search_from_CSP(Search_problem):
    def __init__(self, csp):
        self.csp=csp
        self.variables = list(csp.variables)
        self.variables.sort(key=lambda variable: len(variable.domain))

    def is_goal(self, node):
        return len(node)==len(self.csp.variables)
    
    def start_node(self):
        return {}
    
    def neighbors(self, node):
        var = self.variables[len(node)]
        
        neighbours: list[Arc] = []
        for val in var.domain:
            
            new_assignment = dict_union(node, {var:val})
            if self.csp.consistent(new_assignment):
                neighbours.append(Arc(node, new_assignment))
        return neighbours


csp_problem = grid_to_csp(grid)
search_problem = Backtracking_Search_from_CSP(csp_problem)
dfs = Searcher(search_problem)
dfs.search()

702 paths have been expanded and 7 paths remain in the frontier


{} --> {B8: 5} --> {B8: 5, A1: 9} --> {B8: 5, A1: 9, B4: 1} --> {B8: 5, A1: 9, B4: 1, I8: 9} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8, I4: 4} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8, I4: 4, I1: 6} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8, I4: 4, I1: 6, I2: 2} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8, I4: 4, I1: 6, I2: 2, H9: 6} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8, I4: 4, I1: 6, I2: 2, H9: 6, B5: 4} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8, I4: 4, I1: 6, I2: 2, H9: 6, B5: 4, H6: 1} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8, I4: 4, I1: 6, I2: 2, H9: 6, B5: 4, H6: 1, H5: 3} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8, I4: 4, I1: 6, I2: 2, H9: 6, B5: 4, H6: 1, H5: 3, D2: 3} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8, I4: 4, I1: 6, I2: 2, H9: 6, B5: 4, H6: 1, H5: 3, D2: 3, H3: 8} --> {B8: 5, A1: 9, B4: 1, I8: 9, I5: 7, I6: 8, I4: 4, I1: 6, I2: 2, H9: 6, B5: 4, 

#### Task 4.3

Descibe what do nodes represent in the search tree. Explain your choice of the search algorithm and compare it with any other search algorithm implemented in this assignment. 